In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/data-crime/2018-2021-missing-encoding.csv')

Tiền xử lý dữ liệu

In [ ]:
df['Amount']= df['Incident Description'].str.split(',').str[-1].to_frame(name = 'Amount')
df = df[~df['Amount'].str.contains("[a-zA-Z]").fillna(False)]
df = df[~df['Amount'].str.contains('400')]

In [ ]:
df['time_hour'] = pd.to_datetime(df['Incident Time'], format='%H:%M').dt.hour
df['Incident Month'] = pd.DatetimeIndex(df['Incident Date']).month

In [ ]:
df_18_19_20 = df.loc[(df['Incident Year'] == 2018) | (df['Incident Year'] == 2019) | (df['Incident Year'] == 2020)]
df = df_18_19_20.loc[(df['Incident Category'] == 'Larceny Theft')]

In [ ]:
df = df.dropna(subset=['Intersection'])
df = df.dropna(subset=['CNN'])
df.isnull().sum()


In [ ]:
#Encoding

df["Report Type Code"] = df["Report Type Code"].astype('category')

df["Report Type Code_encoded"] = df["Report Type Code"].cat.codes

df["Report Type Code"] = df["Report Type Code"].astype('object')

df["Report Type Code_encoded"]

df["Report Type Description"] = df["Report Type Description"].astype('category')

df["Report Type Description_encoded"] = df["Report Type Description"].cat.codes

df["Report Type Description"] = df["Report Type Description"].astype('object')

df["Report Type Description_encoded"]


Đặt biến X là các feature quan trọng, Y là biến mục tiêu


In [ ]:
X = df[['Incident Subcategory_encoded' ,'CNN', 'Police District_encoded', 'Supervisor District','Incident Datetime_encoded', 'Neighborhoods','time_hour']]
y = df['Amount']


In [ ]:
Chia tập train và test với tỉ lệ 8:2

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

Chuẩn hóa

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

Tìm tham số tối ưu cho mô hình RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

class_weight= ['balanced','balanced_supsample']
criterion = ['entropy', 'gini']
max_features = ['auto','sqrt','log2']
n_estimators =  list(range(1, 5))

param_grid_RF = {'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}
print(param_grid_RF)

rdf = RandomForestClassifier()
grid_RF = GridSearchCV(rdf, param_grid_RF, cv=10, scoring='accuracy')
grid_RF.fit(X_train, y_train)


In [ ]:
pd.DataFrame(grid_RF.cv_results_)[['mean_test_score', 'std_test_score', 'params']]

Lưu file kết quả

In [ ]:
pd.DataFrame(grid_RF.cv_results_)[['mean_test_score', 'std_test_score', 'params']].to_csv('Parameter_RF_SMOTE.csv')

In ra tham số tốt nhất cho mô hình RandomForest

In [ ]:
print(grid_RF.best_score_)
print(grid_RF.best_params_)

Tìm tham số tối ưu cho mô hình KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']
param_grid_KNN = dict(n_neighbors=k_range, weights=weight_options)
print(param_grid_KNN)

knn = KNeighborsClassifier()
grid_KNN = GridSearchCV(knn, param_grid_KNN, cv=10, scoring='accuracy')
grid_KNN.fit(X_train, y_train)

Lưu file kết quả

In [ ]:
rKNN=pd.DataFrame(grid_KNN.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
rKNN.to_csv('Result_KNN.csv', index=True)

In ra tham số tốt nhất

In [ ]:
print(grid_KNN.best_score_)
print(grid_KNN.best_params_)

Tìm tham số tối ưu cho mô hình LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'sag','saga']
penalty = ['none', 'l1', 'l2', 'elasticnet']
C = [100, 10, 1.0, 0.1, 0.01]
para_lr = dict(C=C,solver=solvers,penalty=penalty)
print(para_lr)

grid_LR = GridSearchCV(lr, para_lr, cv=10,scoring='accuracy')
grid_LR.fit(X_train, y_train)

Lưu file kết quả

In [ ]:
rLR=pd.DataFrame(grid_LR.cv_results_)[['mean_test_score', 'std_test_score', 'params']]
rLR.to_csv('Result_LogisticRegression.csv', index=True)

In ra bộ tham số tốt nhất

In [ ]:
print(grid_LR.best_score_)
print(grid_LR.best_params_)